In [46]:
# If True, running Regen Error Values from Output JSON File
regen_bool = False

# Enter
1. time_th
2. Multiple Gemini Keys ( >= 2)
3. Upload enriched_movies27k.csv

In [2]:
# time_th 1 -> 100 as gen 100 times for 100% users
time_th = 24

In [3]:
# generating user profile for train set
num_top_movies = 25 # num_top_movies top movies for each user
num_top_tags = 60 # num_top_tags top genome tags for both user and movies

In [ ]:
# Mount Drive for saving UserProfile directly into your drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
gen_users_path = "/content/drive/MyDrive/GenUserProfile"

**For full users, list num_top_movies top movies with profiles for LLM to analyze user preference**

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from google import genai
import os
#from google.colab import files
import zipfile
import urllib.request
from pathlib import Path
import requests
import json
import time
#from google.colab import userdata
#from google.genai import types
from tqdm import tqdm
#from google import genai

In [ ]:
#!rm -r fast-traditional-rs
#!git clone https://github.com/duong1124/Duong_Fast.git

rm: cannot remove 'fast-traditional-rs': No such file or directory
Cloning into 'Duong_Fast'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 187 (delta 86), reused 178 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (187/187), 64.80 KiB | 884.00 KiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
#import sys
#sys.path.append('/content/Duong_Fast/duong_fast')

# Dataload and Preprocess

In [8]:
from notebook_helper import LoadPreprocess20M

loader = LoadPreprocess20M()

In [ ]:
df_movies, df_genome_scores, df_tags, df_genome_tags, df_ratings, df_links = loader.dataload(local = True) # for local testing

In [10]:
final_movies, final_genome_scores, final_ratings, tag_mapping = loader.preprocess(df_movies, df_genome_scores, df_tags, df_genome_tags, df_ratings, df_links)

In [12]:
movie_profile_10239_path = "D:\HUST\Lab\movie_profile_10k.csv"

movie_profile_10k = pd.read_csv(movie_profile_10239_path, index_col=0)

movie_profile_10k

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\ACER\AppData\Local\Temp\ipykernel_7684\194046993.py:1: SyntaxWarning: invalid escape sequence '\H'
  movie_profile_10239_path = "D:\HUST\Lab\movie_profile_10k.csv"


,title,genres,actors,director,description
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter,"Led by Woody, Andy's toys live happily in his ..."
2,Jumanji (1995),Adventure|Children|Fantasy,"Robin Williams, Kirsten Dunst, Bradley Pierce",Joe Johnston,When siblings Judy and Peter discover an encha...
3,Grumpier Old Men (1995),Comedy|Romance,"Walter Matthau, Jack Lemmon, Ann-Margret",Howard Deutch,A family wedding reignites the ancient feud be...
4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devine",Forest Whitaker,"Cheated on, mistreated and stepped on, the wom..."
5,Father of the Bride Part II (1995),Comedy,"Steve Martin, Diane Keaton, Martin Short",Charles Shyer,Just when George Banks has recovered from his ...
...,...,...,...,...,...
128488,Wild Card (2015),Crime|Drama|Thriller,"Jason Statham, Michael Angarano, Dominik Garcia",Simon West,When a Las Vegas bodyguard with lethal skills ...
129354,Focus (2015),Comedy|Crime|Drama|Romance,"Will Smith, Margot Robbie, Rodrigo Santoro","Glenn Ficarra, John Requa","Nicky, an accomplished con artist, gets romant..."
129428,The Second Best Exotic Marigold Hotel (2015),Comedy|Drama,"Dev Patel, Judi Dench, Richard Gere",John Madden,As the Best Exotic Marigold Hotel has only a s...


# Taking sample and Train-Test split

In [ ]:
from notebook_helper import SampleRatings

sampler = SampleRatings(
    final_ratings,
    sample_percent = 1.0,
    sample_method = 'first',
    random_state = 42
)

In [ ]:
gen_file, sample_user_ids = sampler.sample_users()

In [ ]:
sampler.find_max_min_movies_rated(gen_file, sample_user_ids)

Max movies rated by one user: 6590, userId: 8405
Min movies rated by one user: 13, userId: 65137


(np.int64(8405), np.int64(65137))

In [ ]:
sampler.find_max_min_users_rated(gen_file)

Max users rated one movie: 67310, movieId: 296
Min users rated one movie: 20, movieId: 49737


(np.int64(296), np.int64(49737))

In [ ]:
rating_train, rating_test = SampleRatings.split_data_ml20m(
    gen_file,
    split_mode='seq-aware',
    test_ratio=0.2,
    random_seed=42
)

In [ ]:
print(f"Number of users in train set: {len(rating_train['userId'].unique())}")
print(f"Number of users in test set: {len(rating_test['userId'].unique())}")
print(f"Number of movies in train set: {len(rating_train['movieId'].unique())}")
print(f"Number of movies in test set: {len(rating_test['movieId'].unique())}")

Number of users in train set: 138493
Number of users in test set: 138493
Number of movies in train set: 10238
Number of movies in test set: 10239


# Building MovieProfile with TMDB

In [26]:
movie_profile_10k

,title,genres,actors,director,description
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles",John Lasseter,"Led by Woody, Andy's toys live happily in his ..."
2,Jumanji (1995),Adventure|Children|Fantasy,"Robin Williams, Kirsten Dunst, Bradley Pierce",Joe Johnston,When siblings Judy and Peter discover an encha...
3,Grumpier Old Men (1995),Comedy|Romance,"Walter Matthau, Jack Lemmon, Ann-Margret",Howard Deutch,A family wedding reignites the ancient feud be...
4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devine",Forest Whitaker,"Cheated on, mistreated and stepped on, the wom..."
5,Father of the Bride Part II (1995),Comedy,"Steve Martin, Diane Keaton, Martin Short",Charles Shyer,Just when George Banks has recovered from his ...
...,...,...,...,...,...
128488,Wild Card (2015),Crime|Drama|Thriller,"Jason Statham, Michael Angarano, Dominik Garcia",Simon West,When a Las Vegas bodyguard with lethal skills ...
129354,Focus (2015),Comedy|Crime|Drama|Romance,"Will Smith, Margot Robbie, Rodrigo Santoro","Glenn Ficarra, John Requa","Nicky, an accomplished con artist, gets romant..."
129428,The Second Best Exotic Marigold Hotel (2015),Comedy|Drama,"Dev Patel, Judi Dench, Richard Gere",John Madden,As the Best Exotic Marigold Hotel has only a s...


In [33]:
from notebook_helper import MovieMetadata

#tmdb_metadata_path = '/content/enriched_movies27k.csv'
tmdb_metadata_path = r"D:\HUST\Lab\UserGenProfile\enriched_movies27k.csv"

tmdb_metadata = MovieMetadata(
    tmdb_metadata_path,
    final_genome_scores,
    df_genome_tags
)

In [42]:
tmdb_metadata.movie_profile

,movieId,title,genres,actors,director
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,2,Jumanji (1995),Adventure|Children|Fantasy,"Robin Williams, Kirsten Dunst, Bradley Pierce,...",Joe Johnston
2,3,Grumpier Old Men (1995),Comedy|Romance,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,5,Father of the Bride Part II (1995),Comedy,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy,"Franz Dinda, Florian Lukas, Oona-Devi Liebich,...",Granz Henman
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,"Rick Kavanian, Axel Stein, Eva Habermann, Chri...",Matthias Dinter
27275,131258,The Pirates (2014),Adventure,"Son Ye-jin, Kim Nam-gil, Oh Dal-su, Park Cheol...",Lee Suk-hoon
27276,131260,Rentun Ruusu (2001),(no genres listed),"Martti Suosalo, Ilkka Koivula, Riitta Salminen...",Timo Koivusalo


In [35]:
tmdb_metadata.check_null()

Column 'movieId': 0 null values
Column 'title': 0 null values
Column 'genres': 0 null values
Column 'actors': 1029 null values
Column 'director': 589 null values


In [36]:
tmdb_metadata.get_data_metadata(1, print_check = True)

Movie ID: 1
Title: Toy Story (1995)
Year: 1995
Genres: Adventure, Animation, Children, Comedy, Fantasy
Actors: Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn
Director: John Lasseter


('Toy Story (1995)',
 'Adventure, Animation, Children, Comedy, Fantasy',
 'Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn',
 'John Lasseter',
 '1995')

Movie Profile as a dict
1. Title
2. Year
3. Genres
4. Actors
5. Directors
6. Top tags with relevance scores

In [37]:
tmdb_metadata.build_movie_profile(1, num_tags = 100)

{'Title': 'Toy Story (1995)',
 'Year': '1995',
 'Genres': 'Adventure, Animation, Children, Comedy, Fantasy',
 'Actors': 'Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn',
 'Directors': 'John Lasseter',
 'Top tags with relevance scores': 'toys : 0.9992, computer animation : 0.9985, pixar animation : 0.9960, kids and family : 0.9908, animation : 0.9858, kids : 0.9792, pixar : 0.9667, children : 0.9643, cartoon : 0.9565, imdb top 250 : 0.9420, animated : 0.9333, childhood : 0.9263, great movie : 0.9207, disney animated feature : 0.9137, friendship : 0.9117, story : 0.9075, light : 0.9000, adventure : 0.8920, original : 0.8793, fun : 0.8778, disney : 0.8730, family : 0.8485, unlikely friendships : 0.8422, good : 0.8355, great : 0.8353, fun movie : 0.8230, cgi : 0.8193, classic : 0.8193, nostalgic : 0.8170, cute : 0.8000, clever : 0.7803, witty : 0.7768, heartwarming : 0.7588, original plot : 0.7580, oscar (best animated feature) : 0.7540, technology : 0.7523, storytelling : 0.

# Movie Profile

In [38]:
from notebook_helper import CleanedMovieProfile

cleaned_movie_profile = CleanedMovieProfile(
    tmdb_metadata_path = tmdb_metadata_path,
    movie_profile_csv = movie_profile_10k,
    df_genome_scores = final_genome_scores,
    df_genome_tags = df_genome_tags,
    df_ratings = final_ratings,
    num_tags = num_top_tags
)

Creating movie paragraphs: 100%|██████████| 10239/10239 [00:02<00:00, 4509.21movie/s]


In [45]:
cleaned_movie_profile.movie_profile

,movieId,title,genres,actors,director
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,2,Jumanji (1995),Adventure|Children|Fantasy,"Robin Williams, Kirsten Dunst, Bradley Pierce,...",Joe Johnston
2,3,Grumpier Old Men (1995),Comedy|Romance,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,5,Father of the Bride Part II (1995),Comedy,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy,"Franz Dinda, Florian Lukas, Oona-Devi Liebich,...",Granz Henman
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,"Rick Kavanian, Axel Stein, Eva Habermann, Chri...",Matthias Dinter
27275,131258,The Pirates (2014),Adventure,"Son Ye-jin, Kim Nam-gil, Oh Dal-su, Park Cheol...",Lee Suk-hoon
27276,131260,Rentun Ruusu (2001),(no genres listed),"Martti Suosalo, Ilkka Koivula, Riitta Salminen...",Timo Koivusalo


In [ ]:
cleaned_movie_profile.cleaned_movie_profile

,ProfileParagraph
2,Title: Jumanji (1995). Year: 1995. Genres: Adv...
29,"Title: City of Lost Children, The (Cité des en..."
32,Title: Twelve Monkeys (a.k.a. 12 Monkeys) (199...
47,Title: Seven (a.k.a. Se7en) (1995). Year: 1995...
50,"Title: Usual Suspects, The (1995). Year: 1995...."
...,...
8913,Title: Around the Bend (2004). Year: 2004. Gen...
117364,Title: Virunga (2014). Year: 2014. Genres: Doc...
95193,Title: Pocahontas II: Journey to a New World (...
114459,Title: White Bird in a Blizzard (2014). Year: ...


# Generate UserProfile

In [ ]:
from notebook_helper import GenUserProfile

user_gen = GenUserProfile(
    tmdb_metadata_path,
    rating_train,
    final_genome_scores,
    df_genome_tags,
    num_top_movies = num_top_movies,
    num_top_tags = num_top_tags,
    time_th = time_th,
    api_keys = MY_GEMINI_KEYS,
    gen_users_path = gen_users_path
)

**Take a double check to your desired userId to gen here**

In [ ]:
user_gen.all_userId_gen()

4_th time: [4153 4154 4155 ... 5534 5535 5536] 
14_th time: [17993 17994 17995 ... 19374 19375 19376] 
24_th time: [31833 31834 31835 ... 33214 33215 33216] 
34_th time: [45673 45674 45675 ... 47054 47055 47056] 
44_th time: [59513 59514 59515 ... 60894 60895 60896] 
54_th time: [73353 73354 73355 ... 74734 74735 74736] 
64_th time: [87193 87194 87195 ... 88574 88575 88576] 
74_th time: [101033 101034 101035 ... 102414 102415 102416] 
84_th time: [114873 114874 114875 ... 116254 116255 116256] 
94_th time: [128713 128714 128715 ... 130094 130095 130096] 


In [ ]:
user_gen.time_th

24

In [ ]:
print(user_gen.split_userId_train())
print(len(user_gen.split_userId_train()))

[31833 31834 31835 ... 33214 33215 33216]
1384


In [ ]:
user_profiles = user_gen.build_user_profiles(nap = False)

Building user profiles:   0%|          | 6/1384 [00:31<2:00:37,  5.25s/user]


KeyboardInterrupt: 

In [ ]:
user_gen.save_user_profiles(user_profiles)

In [ ]:
from notebook_helper import UserProfileTools

user_check = UserProfileTools(user_profiles)

In [ ]:
user_check.visualize_user_profile(print_out = True)

In [ ]:
user_check.check_user_profile()

# Test CleanedUserProfile

In [47]:
user_profile_path = r"D:\HUST\Lab\UserGenProfile\user_profile_13k.json"
user_profile_13k = pd.read_json(user_profile_path)

In [49]:
from notebook_helper import CleanedUserProfile

cleaned_user_profile = CleanedUserProfile(
    user_profile_13k
)

Creating profile paragraphs: 100%|██████████| 13849/13849 [00:00<00:00, 32838.10user/s]


In [50]:
cleaned_user_profile.cleaned_user_profile

,ProfileParagraph
24,"FavoriteDecade: 1990. FavoriteGenres: Comedy, ..."
36,"FavoriteDecade: 1990. FavoriteGenres: Action, ..."
71,"FavoriteDecade: 2010. FavoriteGenres: Drama, C..."
87,"FavoriteDecade: 1990. FavoriteGenres: Crime, T..."
88,"FavoriteDecade: 1990. FavoriteGenres: Drama, C..."
...,...
138449,"FavoriteDecade: 1980. FavoriteGenres: Action, ..."
138472,"FavoriteDecade: 1990. FavoriteGenres: Drama, C..."
138476,FavoriteDecade: 2000. FavoriteGenres: Adventur...
138487,"FavoriteDecade: 1990. FavoriteGenres: Drama, R..."


In [52]:
cleaned_user_profile.print_a_profile

<bound method CleanedUserProfile.print_a_profile of                                          ProfileParagraph
24      FavoriteDecade: 1990. FavoriteGenres: Comedy, ...
36      FavoriteDecade: 1990. FavoriteGenres: Action, ...
71      FavoriteDecade: 2010. FavoriteGenres: Drama, C...
87      FavoriteDecade: 1990. FavoriteGenres: Crime, T...
88      FavoriteDecade: 1990. FavoriteGenres: Drama, C...
...                                                   ...
138449  FavoriteDecade: 1980. FavoriteGenres: Action, ...
138472  FavoriteDecade: 1990. FavoriteGenres: Drama, C...
138476  FavoriteDecade: 2000. FavoriteGenres: Adventur...
138487  FavoriteDecade: 1990. FavoriteGenres: Drama, R...
138489  FavoriteDecade: 1990. FavoriteGenres: Drama, C...

[13849 rows x 1 columns]>

# Regen Error Values from Output JSON File

Chỉ chạy khi gen xong một sample và muốn kiểm tra xem có những id nào bị lỗi để gen lại.

In [ ]:
error_json_path = "/content/drive/MyDrive/GenUserProfile/error_json"
if regen_bool:
  df = pd.read_json(error_json_path, orient='index')
  userId_gen = GenUserProfile.detect_json_errors(df,error_json_path)